In [52]:
import serial
import serial.tools.list_ports
import time
import tkinter as tk
from tkinter import ttk
import threading

class SerialGUI:
    def __init__(self, master):
        self.master = master
        self.master.title("Serial Communication Tool")

        self.baud_rates = [9600, 19200, 38400, 57600, 115200]

        self.create_widgets()

    def create_widgets(self):
        self.frame1 = ttk.Frame(self.master)
        self.frame1.pack(side='top', anchor='w', padx=10, pady=10)

        self.port_label = ttk.Label(self.frame1, text="Serial Port:")
        self.port_label.grid(row=0, column=0, padx=(0, 5), sticky='w')

        self.port_combobox = ttk.Combobox(self.frame1, state="readonly")
        self.port_combobox.grid(row=0, column=1, sticky='w')
        self.port_combobox["values"] = scan_serial_ports()

        self.baud_label = ttk.Label(self.frame1, text="Baud Rate:")
        self.baud_label.grid(row=0, column=2, padx=(10, 5), sticky='e')

        self.baud_combobox = ttk.Combobox(self.frame1, state="readonly")
        self.baud_combobox.grid(row=0, column=3, sticky='w')
        self.baud_combobox["values"] = self.baud_rates

        self.connect_button = ttk.Button(self.frame1, text="Connect", command=self.connect)
        self.connect_button.grid(row=0, column=4, padx=(10, 0))

        self.frame2 = ttk.Frame(self.master)
        self.frame2.pack(side='top', anchor='w', padx=10, pady=10)

        self.send_label = ttk.Label(self.frame2, text="Send Command:")
        self.send_label.grid(row=0, column=0, padx=(0, 5), sticky='w')

        self.command_entry = ttk.Entry(self.frame2, width=30)
        self.command_entry.grid(row=0, column=1, sticky='w')

        self.send_button = ttk.Button(self.frame2, text="Send", command=self.send_command)
        self.send_button.grid(row=0, column=2, padx=(10, 0), sticky='w')
        
        self.send_data_type_label = ttk.Label(self.frame2, text="Send Data Type:")
        self.send_data_type_label.grid(row=1, column=0, padx=(0, 5), pady=(5, 0), sticky='w')

        self.send_data_type_combobox = ttk.Combobox(self.frame2, state="readonly")
        self.send_data_type_combobox.grid(row=1, column=1, pady=(5, 0), sticky='w')
        self.send_data_type_combobox["values"] = ("Text", "Hex")
        self.send_data_type_combobox.current(0)

        self.receive_data_type_label = ttk.Label(self.frame2, text="Receive Data Type:")
        self.receive_data_type_label.grid(row=2, column=0, padx=(0, 5), pady=(5, 0), sticky='w')

        self.receive_data_type_combobox = ttk.Combobox(self.frame2, state="readonly")
        self.receive_data_type_combobox.grid(row=2, column=1, pady=(5, 0), sticky='w')
        self.receive_data_type_combobox["values"] = ("Text", "Hex")
        self.receive_data_type_combobox.current(0)

        self.frame3 = ttk.Frame(self.master)
        self.frame3.pack(side='top', anchor='w', padx=10, pady=10)

        self.response_label = ttk.Label(self.frame3, text="Response:")
        self.response_label.grid(row=0, column=0, padx=(0, 5), sticky='w')

        self.response_text = tk.Text(self.frame3, width=40, height=10)
        self.response_text.grid(row=0, column=1, sticky='w')
        

        # 
        self.frame4 = ttk.Frame(self.master)
        self.frame4.pack(side='top', anchor='w', padx=10, pady=10)

        self.multi_command_label = ttk.Label(self.frame4, text="Commands (one per line):")
        self.multi_command_label.grid(row=0, column=0, padx=(0, 5), sticky='nw')

        self.multi_command_text = tk.Text(self.frame4, width=20, height=5, wrap='word')
        self.multi_command_text.grid(row=0, column=1, sticky='w')

        self.delay_label = ttk.Label(self.frame4, text="Delay (seconds):")
        self.delay_label.grid(row=1, column=0, padx=(0, 5), pady=(5, 0), sticky='e')

        self.delay_entry = ttk.Entry(self.frame4, width=8)
        self.delay_entry.grid(row=1, column=1, pady=(5, 0), sticky='w')

        self.start_multi_button = ttk.Button(self.frame4, text="Start", command=self.start_multi_commands)
        self.start_multi_button.grid(row=1, column=2, padx=(10, 0), pady=(5, 0), sticky='w')

        self.stop_multi_button = ttk.Button(self.frame4, text="Stop", command=self.stop_multi_commands)
        self.stop_multi_button.grid(row=1, column=3, padx=(10, 0), pady=(5, 0), sticky='w')


    def connect(self):
        if not hasattr(self, "ser") or not self.ser.isOpen():
            self.port = self.port_combobox.get()
            self.baud_rate = int(self.baud_combobox.get())

            try:
                self.ser = serial.Serial(self.port, self.baud_rate, timeout=1)
                self.response_text.insert(
                    tk.END, f"Connected to {self.port} at {self.baud_rate} baud.\n")
                self.connect_button.config(text="Disconnect", command=self.disconnect)
            except Exception as e:
                self.response_text.insert(tk.END, f"Error: {e}\n")
        else:
            self.disconnect()

    def disconnect(self):
        if hasattr(self, "ser") and self.ser.isOpen():
            try:
                self.ser.close()
                self.response_text.insert(tk.END, f"Disconnected from {self.port}.\n")
                self.connect_button.config(text="Connect", command=self.connect)
            except Exception as e:
                self.response_text.insert(tk.END, f"Error: {e}\n")
    
    def send_command(self):
        if command is None:
            command = self.command_entry.get()

        if not command:
            self.response_text.insert(tk.END, "Please enter a command.\n")
            return

        if not hasattr(self, "ser") or not self.ser.isOpen():
            self.response_text.insert(
                tk.END, "Please connect to a serial port first.\n")
            return

        send_data_type = self.send_data_type_combobox.get()
        receive_data_type = self.receive_data_type_combobox.get()

        try:
            if send_data_type == "Text":
                self.ser.write((command + '\r\n').encode())
            elif send_data_type == "Hex":
                command_hex = bytes.fromhex(command)
                self.ser.write(command_hex)

            time.sleep(1)
            response = self.ser.readline()

            if receive_data_type == "Text":
                response = response.decode('utf-8').strip()
            elif receive_data_type == "Hex":
                response = response.hex()

            self.response_text.insert(
                tk.END, f">>> Sent: {command}\n<<< Received: {response}\n")
        except Exception as e:
            self.response_text.insert(tk.END, f"Error: {e}\n")
   
    def start_multi_commands(self):
        commands = self.multi_command_text.get(1.0, tk.END).strip().split('\n')
        delay = float(self.delay_entry.get())
        self.stop_multi_commands_flag = False
        self.multi_command_thread = threading.Thread(target=self.send_multi_commands, args=(commands, delay))
        self.multi_command_thread.start()

    def send_multi_commands(self, commands, delay):
        command_index = 0
        while not self.stop_multi_commands_flag:
            command = commands[command_index]
            self.send_command(command)
            time.sleep(delay)
            command_index += 1
            if command_index >= len(commands):
                command_index = 0

    def stop_multi_commands(self):
        self.stop_multi_commands_flag = True
        if hasattr(self, "multi_command_thread"):
            self.multi_command_thread.join()

    def scan_serial_ports():
        ports = serial.tools.list_ports.comports()
        available_ports = []

        for port in ports:
            available_ports.append(port.device)

        return available_ports




In [53]:
root = tk.Tk()
gui = SerialGUI(root)
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\HEQ\AppData\Local\Temp\ipykernel_42256\3879953526.py", line 124, in send_command
    if command is None:
UnboundLocalError: local variable 'command' referenced before assignment
